# This script is mimicking what the backend would be doing when running an UDF

In [7]:
from os import getcwd
getcwd()

'/home/pzellner@eurac.edu/git_projects/r4openeo-usecases/uc2-ts-breakdetection/local_udf'

In [8]:
from openeo_r_udf.udf_lib import prepare_udf, execute_udf
import time
import xarray as xr

In [12]:
# config
load_file = "../00_data_local/test.nc"
udf_file = "../00_udfs/bfast_udf.R"
save_file = "./results/local_udf_test.tif"
dimension = "t"
context = {'start_monitor': 2018}

In [14]:
# load and prepare data - depending on your data structure
dataset = xr.load_dataset(load_file)
# dataset = dataset.drop('transverse_mercator') # ('crs')
# data = dataset.to_array(dim = 'var')
# data = data.squeeze('var')
# data = data.drop('var')
# data['t'] = data.t.astype('str')

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'h5netcdf']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
http://xarray.pydata.org/en/stable/getting-started-guide/installing.html
http://xarray.pydata.org/en/stable/user-guide/io.html

In [ ]:
# define runer for udf
def run(process, udf, dimension = None, context = None):
    # Run UDF executor
    t1 = time.time() # Start benchmark
    print('start: ', udf) #t1
    result = execute_udf(process, udf, data, dimension = dimension, context = context, parallelize = parallelize, chunk_size = chunk_size)
    t2 = time.time() # End benchmark
    print('end: ', udf) #t2

    # Print result and benchmark
    print('  Time elapsed: %s' % (t2 - t1))
    #save_result(result, save_file)

In [ ]:
print('reduce_dimension bfast')
run('reduce_dimension', udf_file, dimension = dimension, context = context)